In [ ]:
!pip install tensorflow-gpu==2.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.9 MB 2.1 MB/s 
     |████████████████████████████████| 578.0 MB 15 kB/s 
     |████████████████████████████████| 438 kB 45.2 MB/s 
     |████████████████████████████████| 1.7 MB 38.2 MB/s 
     |████████████████████████████████| 5.9 MB 26.2 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninsta

In [ ]:
import tensorflow_text as text
import tensorflow_hub as hub
import tensorflow as tf
import pandas as pd

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model


In [ ]:
df = pd.read_csv("processed_data.csv")
df.head(5)

,Review,Sentiment
0,book horrible possible rate lower star avoid r...,0
1,amazon reviews purchasing books especially ale...,0
2,book horrible possible rate lower star avid re...,0
3,whos writing reviews read repitition repititio...,0
4,picked book series eyre affair based purely pr...,0


In [ ]:
df['Review'] = df['Review'].astype(str)

In [ ]:
df.groupby('Sentiment').describe()

Review                                                            \
           count unique                                                top   
Sentiment                                                                    
0           5000   4859  agree previous review phone cord worked retrac...   
1           5000   4969  kitchenaid professional kvgx great product buy...   

                
          freq  
Sentiment       
0            5  
1            3

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['Review'],df['Sentiment'], stratify=df['Sentiment'])

In [ ]:
X_train.head(4)

964     hero coldmean brute deserve gentle wife kassia...
941     book deals fairly rediculous topics mysticism ...
808     doubt mind josh harris person sincerely partic...
8332    dudes stop making kinds records cuz hip hop ra...
Name: Review, dtype: object

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [ ]:
len(X_train)

7500

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=50)

Epoch 1/50
235/235 [==============================] - 83s 312ms/step - loss: 0.6911 - accuracy: 0.5512
Epoch 2/50
235/235 [==============================] - 77s 327ms/step - loss: 0.6492 - accuracy: 0.6193
Epoch 3/50
235/235 [==============================] - 77s 327ms/step - loss: 0.6250 - accuracy: 0.6577
Epoch 4/50
235/235 [==============================] - 77s 326ms/step - loss: 0.6166 - accuracy: 0.6611
Epoch 5/50
235/235 [==============================] - 77s 326ms/step - loss: 0.6061 - accuracy: 0.6691
Epoch 6/50
235/235 [==============================] - 77s 327ms/step - loss: 0.5935 - accuracy: 0.6871
Epoch 7/50
235/235 [==============================] - 77s 326ms/step - loss: 0.5876 - accuracy: 0.6837
Epoch 8/50
235/235 [==============================] - 77s 326ms/step - loss: 0.5830 - accuracy: 0.6896
Epoch 9/50
235/235 [==============================] - 77s 326ms/step - loss: 0.5778 - accuracy: 0.6985
Epoch 10/50
235/235 [==============================] - 77s 326ms/step - l

In [ ]:
model.evaluate(X_test, y_test)

79/79 [==============================] - 26s 321ms/step - loss: 0.5731 - accuracy: 0.7020


[0.5730840563774109, 0.7020000219345093]

In [ ]:
test_predictions = (model.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, test_predictions))

79/79 [==============================] - 27s 339ms/step
              precision    recall  f1-score   support

           0       0.65      0.88      0.75      1250
           1       0.82      0.52      0.64      1250

    accuracy                           0.70      2500
   macro avg       0.73      0.70      0.69      2500
weighted avg       0.73      0.70      0.69      2500



In [ ]:
model.save('bert.h5')
print('Model Saved!')

Model Saved!


In [ ]:
!cp /content/drive/MyDrive/bert.h5  /content

In [ ]:
savedModel=load_model('bert.h5',custom_objects={'KerasLayer':hub.KerasLayer})
savedModel.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [ ]:
list = []
reviews = "The product is good"
list.append(reviews)
sentiment = (savedModel.predict(list) > 0.5).astype(int)
if sentiment == 1 :
  print("Positive")

else:
  print("Negative")

1/1 [==============================] - 1s 591ms/step
Positive


In [ ]:
!cp /content/bert.h5 /content/drive/MyDrive